# 常用基本计算

在日常科研学习工作中有一些计算，比如统计计算，是很常遇到的，这里日常积累做些记录以备后用。

## NSE

NSE 全名 Nash–Sutcliffe model efficiency coefficient，用来评价水文模型的预测能力，原文可以参考：[River flow forecasting through conceptual models part I — A discussion of principles](https://www.sciencedirect.com/science/article/pii/0022169470902556)。这里根据维基百科[Nash–Sutcliffe model efficiency coefficient](https://en.wikipedia.org/wiki/Nash%E2%80%93Sutcliffe_model_efficiency_coefficient)做一些记录。

先上公式：
$$NSE=1-\frac{\sum _{t=1}^{T} (Q_o^t - Q_m^t)^2}{\sum _{t=1}^T (Q_o^t -\overline{Q_o})^2}$$
$Q_o$是观测径流平均值，$Q_m$是模拟径流。$Q_o^t$是t时段径流。

NSE值范围$-\infty$至1，NSE=1 表示模拟值和观测值非常匹配。NSE=0则表明模型预测和观测均值一致。而NSE为负数则说明观测均值都比模拟值效果好，即模拟与观测的residual variance 比 径流本身的方差还大。总之，越接近1，说明预测越好。0.5<NSE<0.65可以算是一个足够好的预测NSE的范围。

当模拟是回归时，NSE和$R^2$等价。

NSE对极值比较敏感，因此离群值影响较大。

纳什效率可以用来定量描述模型输出的准确性，而不仅仅是流量。只要有观测数据，并与模型结果进行比较，该指标可用于描述其他模型的预测精度。

下面给出二维的NSE计算方式。每行表示不同的时间，每列表示不同的站点。

In [3]:
import numpy as np
sim = np.random.rand(6,4)
sim

array([[0.72209182, 0.21352318, 0.62410322, 0.389707  ],
       [0.89507492, 0.75888337, 0.34669306, 0.40842543],
       [0.14654104, 0.46812356, 0.13435416, 0.76889576],
       [0.31079085, 0.84641048, 0.76140257, 0.94652568],
       [0.09716227, 0.27537252, 0.67535365, 0.50391768],
       [0.87018306, 0.82371468, 0.15580497, 0.22831284]])

In [4]:
obs = np.random.rand(6,4)
obs

array([[0.55331267, 0.48772107, 0.26298621, 0.86993086],
       [0.83876898, 0.31892878, 0.07505383, 0.47556347],
       [0.91504126, 0.53631456, 0.79054552, 0.52907441],
       [0.00748106, 0.1855167 , 0.32326133, 0.69029544],
       [0.50406595, 0.48948671, 0.73682538, 0.59250742],
       [0.16226021, 0.92901116, 0.45750459, 0.05923551]])

先求每个站点的观测均值，注意尽量不要用循环，for会使运算变慢。

In [6]:
obs_mean=np.mean(obs,axis=0)
obs_mean

array([0.49682169, 0.49116316, 0.44102948, 0.53610118])

然后扩展为同样shape的数组，以做减法运算（也可以广播）

In [7]:
obs_means=np.tile(obs_mean, (obs.shape[0],1))
obs_means

array([[0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118],
       [0.49682169, 0.49116316, 0.44102948, 0.53610118]])

接着做减法并求平方，然后每列各自求和：

In [16]:
denominator=np.sum((obs-obs_means)**2, axis=0)
denominator

array([0.64646491, 0.31684862, 0.38943503, 0.36951484])

分子部分先计算obs和sim的差值，然后平方，并每列各自求和：

In [17]:
numerator = np.sum((obs-sim)**2, axis=0)
numerator

array([1.38097153, 0.76710736, 0.92154964, 0.39472597])

然后是element-wise相除：

In [18]:
quotient = numerator/denominator
quotient

array([2.13618948, 2.42105318, 2.36637586, 1.06822765])

构造一个1向量，然后相减即可

In [21]:
my_ones=np.ones(obs.shape[1])
NSE=my_ones-quotient
NSE

array([-1.13618948, -1.42105318, -1.36637586, -0.06822765])

以上就是NSE的计算方法。

接下来讨论一个实际情况，在实际观测中我们可能会有缺失值，此时完全按照上述方法计算可能会报错，所以需要处理下特殊情况。

In [2]:
import numpy as np
a1=np.array([[1,2,3,4,5],[6,7,8,9,np.nan],[10,11,np.nan,12,13]])
a1

array([[ 1.,  2.,  3.,  4.,  5.],
       [ 6.,  7.,  8.,  9., nan],
       [10., 11., nan, 12., 13.]])

In [3]:
a2=np.arange(15).reshape(3,5)
a2

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14]])

遇到nan的值，应该将它们排除，对应地也将模拟值排除，然后再逐个计算即可。不过这样就不好做向量化运算了，某些情况下可能会非常慢，所以想想另一种办法。先试试对应位置都设置为0会如何。

In [10]:
def nse_1d(obs,sim):
    obs_mean=np.mean(obs)
    SST=np.sum((obs-obs_mean)**2)
    SSRes = np.sum((obs -sim) ** 2)
    nse = 1 - SSRes / SST
    return nse

a_obs=np.array([1,2])
a_sim=np.array([3,4])
print(nse_1d(a_obs,a_sim))

-15.0


In [11]:
a_obs=np.array([1,2,0])
a_sim=np.array([3,4,0])
print(nse_1d(a_obs,a_sim))

-3.0


从上面结果看，这种方式是不可行的。那试试换成均值？

In [12]:
a_obs=np.array([1,2,1.5])
a_sim=np.array([3,4,3.5])
print(nse_1d(a_obs,a_sim))

-23.0


In [13]:
a_sim=np.array([3,4,np.nan])
a_sim_avg=np.mean(a_sim)
a_sim_avg

nan

换成均值也不行，其实对于一维的情况，我们可以直接cut掉

## 彭曼公式

关于彭曼公式的基本资料可以参考联合国粮农组织的文档介绍：[Crop evapotranspiration - Guidelines for computing crop water requirements - FAO Irrigation and drainage paper 56](http://www.fao.org/3/X0490E/x0490e00.htm#Contents)。这里简要翻译下其中主要内容。

FAO的这个蒸散发计算主要是针对农作物的，不过概念都是普适的。首先是对 Evapotranspiration process 的介绍。

### Evapotranspiration process

evapotranspiration （ET）包括两个部分，其一是 evaporation，表示土壤表面水分蒸发损失，另一部分是 transpiration ，是指crop的水分散失，两者一起称为 evapotranspiration 。

evaporation 是 水分由液转气，并从蒸发表面散去的过程，比如湖面，河面，道路，土壤和植被等。蒸发吸热，因为分子需要能量才能逃逸。直接的太阳辐射，还有环境气温提供了这种能量。蒸发面的水汽压差是蒸发面的水汽压与周围大气的水汽压差。随着蒸发的进行，周围的空气逐渐饱和，蒸发过程将会减慢，如果潮湿的空气不转移到大气中，这一过程可能会停止。当蒸发面为土壤表面时，作物冠层的遮荫程度和蒸发面可用水量是影响蒸发过程的其他因素。频繁的降雨、灌溉从浅水表向土壤上方输送，会湿润土壤表面。在土壤能够快速供水以满足蒸发需求的地方，土壤的蒸发仅由气象条件决定。然而，当降雨和灌溉之间的间隔变大，土壤将水分输送到表面的能力变弱时，表层土壤的含水量下降，土壤表面变干。在这种情况下，有限的水的可用性对土壤蒸发产生控制作用。在土壤表面没有水供应的情况下，蒸发会迅速减少，并可能在几天内几乎完全停止。

transpiration 包括蒸发植物组织中含有的液态水并向大气中排放水蒸气。作物主要通过气孔失去水分。气孔是植物叶子上的小开口，气体和水蒸气会通过这些小开口。水分和一些营养物质被植物的根部吸收，并通过植物运输。蒸发发生在叶片内部，即细胞间隙内，与大气的蒸汽交换由气孔孔控制。几乎所有的水分都通过蒸腾作用流失了，只有一小部分在植物内部被利用。蒸腾作用和直接蒸发一样，取决于能量供应、蒸汽压力梯度和风。因此，在评估蒸腾作用时，应考虑辐射、气温、空气湿度和风的条件。土壤含水量和土壤将水分输送到根系的能力也决定了蒸腾速率，涝渍和土壤水盐度也是如此。蒸腾速率还受作物特性、环境因素和栽培措施的影响。不同种类的植物有不同的蒸腾速率。在评价蒸腾作用时，不仅要考虑作物的类型，还要考虑作物的发育、环境和管理。

蒸发和蒸腾同时发生，这两个过程很难区分。除了表层土壤的水分有效性外，作物土壤的蒸发主要取决于到达土壤表面的太阳辐射的比例。随着作物的生长和作物冠层遮挡越来越多的地面面积，这一比例在生长过程中逐渐降低。**当作物很小的时候，水分主要是由于土壤蒸发而损失的，但是一旦作物生长良好并完全覆盖了土壤，蒸腾作用就成为主要过程**，在播种时，近100%的ET来自蒸发，而在作物全覆盖时，90%以上的ET来自蒸腾。

ET rate 单位通常是 mm/unit time ，rate表示的是单位时间的蒸散发水深当量。举例，对于一个 10000 m2 的面积，1mm/day就是 1000 m3/ha/day。水深也可以单位面积接收的能量表示，该能量是指蒸散发所需的热能，称作 the latent heat of vaporization ($\lambda$)，是水温的函数。比如 20摄氏度对应 $\lambda$ 是 2.45 MJ/kg 。也就是说蒸发1kg的水需要2.45MJ能量。因此，可以一平方米 2.45MJ能量的输入可以蒸发1mm的water，所以1mm 水等价于 2.45MJ/m2 .以 MJ/m2/day 表示的ET rate 由 $\lambda _{ET}$ 潜热通量the latent heat flux 表示。

影响 Evapotranspiration 的因素通常有：Weather parameters, crop characteristics, management and environmental aspects

主要的天气参数有  radiation, air temperature, humidity and wind speed.  大气蒸发能力由The evaporation power of the atmosphere 由 the reference crop evapotranspiration ($ET_0$) 表示. $ET_0$ 代表了标准植被表面的蒸散发。

The crop type, variety and development stage 是考虑植被蒸散发的关键因素。标准条件下的作物蒸散量($ET_c$)是指在土壤水分适宜、管理和环境条件良好的条件下，大面积种植作物，在一定的气候条件下达到充分生产所需的蒸发量。

土壤盐碱度、土地肥力差、肥料施用有限、土层坚硬或不能渗透、病虫害防治缺乏以及土壤管理不善等因素可能限制作物的生长，减少蒸散发。在评估ET时需要考虑的其他因素包括地面覆盖、植物密度和土壤含水量。土壤含水量对ET的影响主要取决于水分亏缺的大小和土壤类型。另一方面，过多的水分会导致涝渍，可能会损害根系，通过抑制呼吸来限制根系对水分的吸收。在评估ET率时，应额外考虑影响ET过程的气候和作物因素的管理措施范围。耕作方式和灌溉方式可以改变小气候，影响作物特性或影响土壤和作物表面的湿润。防风林可以降低风速，并直接降低ET率。这种影响可能是显著的，特别是在多风、温暖和干燥的条件下，虽然树木本身的蒸发蒸腾可能抵消了田间的任何减少。在年轻果园里，树木间距很大，可以使用精心设计的滴灌或滴灌系统来减少土壤蒸发。滴管直接将水喷洒在树木附近的土壤上，从而使土壤表面的主要部分保持干燥，并限制蒸发损失。使用地膜，特别是当作物很小的时候，是另一种大幅度减少土壤蒸发的方法。抗蒸腾剂，如封闭气孔、成膜或反射物质，可减少作物的水分损失，从而降低蒸腾速率。

### Methods of estimating potential and actual evaporation

本节主要参考了[Methods of estimating potential and actual evaporation](https://www.researchgate.net/file.PostFileLoader.html?id=58b6feb8f7b67e3dc71ca3c1&assetKey=AS%3A467146313605122%401488387768864)（注意点击即下载了）。水文上通常关注两个关于蒸发的变量， potential evapotranspiration (PE) 和 实际蒸散发值 ET。PE 就是指供水充足时的最大蒸发能力。ET 实际上并不容易测量，一个常见的测量方式是用  irrigated lysimeter ，用它可以测量PE。

PE = Rainfall + Irrigation - Percolation

还有就是蒸发皿，通过皿中水位变化可以估算实际蒸发量。还有就是其他一些测量方式，这里不再赘述。

通常要得到流域的ET会先计算PE。最常用的计算PE的公式就是Penman equation，一个计算**开放水域的蒸散发估计**的公式：
$$PE=(\frac{\triangle}{\triangle + \gamma})*Q_{ET}+(\frac{\gamma}{\triangle + \gamma})*E_{at} \ \ \ \ \   (mm/day)$$
其中，
$$Q_{ET}=Q_S*(1-r)-Q_1$$
$$Q_1= 0.95*[8.64*10^7/(\rho*\lambda)]*\sigma*(273.16+Ta)^4*(0.53+0.065*(e_d-1.0)^{1/2})*(0.10+0.90*(n/N))$$
$$E_{at} = 0.3*(1+0.5*u_2)*(e_a-e_d) $$

- $\triangle$ (mb/C)是饱和蒸汽压力曲线相对于温度的斜率
- $\gamma$是hygrometric常数=0.65 mb/C 
- $Q_1$是水体长波辐射
- r是植被覆盖系数 (r = 0.25 for a short grassed surface)
- $T_a$是气温（C）
- n/N 是实际/可能 日照小时数
- $\rho$是水密度(kg/m3)
- $\lambda$是水蒸发潜热（J/kg）
- $\sigma$是 Stefan Bolzman 常数 (= $5.7*10^{-8} W/(m^2*grad^4)$). 
- $u_2$是风速（m/s）
- $e_a$是测得的空气温度的饱和蒸汽压
- $e_b$是空气的实际蒸汽压(mb)

注：$Q_{ET}, Q_s, Q_l, E_{at}$ 单位都是 mm/day. 

彭曼公式是基于地表能量平衡公式： $R_n=LE+G+H$ ，更多内容可以参考：https://zhuanlan.zhihu.com/p/59166965

具体的推导过程可以参考：https://edepot.wur.nl/183107

如果觉得不好理解，可以把热通量类比电流，压差或温差类比电压，resistance就是电阻，欧姆定律就会好理解一些了。

另外，饱和蒸汽下，两个平面压差和温差之间的关系就是$\triangle$，温压间关系可见图5.5。

然后注意式5.15分子分母写反了，不过后面的是对的。

### Penman-Monteith equation

接下来记录一种从**气象数据中标准化计算参考蒸散发**的方法。FAO 推荐 将Penman-Monteith方法作为确定参考蒸散发ETo的唯一方法。这里简单给出该方法及其推导过程，所需的气象资料以及相应的参考面定义。

1948年，Penman将能量平衡和质量转移方法结合起来，从阳光、温度、湿度和风速的标准气候记录中推导出一个公式来计算开阔水面的蒸发。这种组合方法由许多研究人员进一步发展，并通过引入阻力因素扩展到**作物表面**。

cropped表面和水面的主要不同：

- 作物表面(例如，0.23)和水面(0.05 - 0.07)的反照率(或太阳辐射的反射系数)是不同的
- 作物表面有粗糙度(取决于作物高度和风速)，因此空气动力阻力(ra)与水面有很大的不同。

具体的推导过程可以继续参考上一小节参考的文章。

首先是wet crop的蒸发计算，和前面的式子比较，主要是换了$E_a$的表达。

然后是dry crop，就是这里的Penman-Monteith equation了，$E_a$的表达进一步变化，主要是提到的“big leaf”的概念要理解下。然后串联电阻，电流一样，可以简单理解式5.25。

术语阻力区分了空气动力阻力和表面阻力因子(下图)。表面阻力参数通常合并为一个参数，即“bulk”表面阻力参数，它与空气动力阻力是串联的。

![](x0490e07.jpg)

表面阻力(rs)描述了水蒸气通过气孔、总叶面积和土壤表面的阻力。空气动力阻力，ra，描述了来自植被向上的阻力和来自空气流过植被表面的摩擦。虽然植被层的交换过程过于复杂，不能完全用这两个阻力因子来描述，但是在测量和计算的蒸散发速率之间可以得到很好的相关性，特别是在均匀的草参考面。

Penman-Monteith form of the combination equation :
$$\lambda_{ET}=\frac{\triangle (R_n-G)+\rho_a c_p \frac{e_s-e_a}{r_a}}{\triangle +\gamma (1+\frac{r_s}{r_a})}$$

其中，$R_n$ 是净辐射, G 是土壤热通量, $(e_s - e_a)$ 表示了空气的蒸汽压不足, $r_a$ 是恒定气压下的空气密度, $c_p$ 是空气指定热量, D 表示饱和蒸汽压温度关系斜率, g 是psychrometric 常数, $r_s$ 和 $r_a$ 是 （bulk) 表面和气动力阻力.。

如上所述，Penman-Monteith方法包括了控制能量交换和相应的潜热通量(蒸散发)的所有参数。大部分的参数都是测量出来的，或者可以很容易地从天气数据中计算出来。该方程可以直接计算任意作物的蒸散发，因为作物的表面和空气阻力是作物特有的。